#Load

In [41]:
from pathlib import Path
import pandas as pd
import urllib.request


def load_housing_data():
  sold_file=Path("datasets/sold_melbourne.csv")
  if not sold_file.is_file():
    url="https://raw.githubusercontent.com/mmercan/handson-ml3/refs/heads/main/data/housing/sold_melbourne.csv"
    urllib.request.urlretrieve(url,sold_file)
  return pd.read_csv(sold_file)


housing = load_housing_data()

housing['price']=housing['price'].str.replace('$', '', regex=False).astype(float)

#housing['price']=housing['price'].replace('[\$,]', '', regex=True).astype(float)

# housing['price_numeric'] = housing['price'].apply(
#     lambda x: float(str(x).replace('$', '').replace(',', '')) if pd.notnull(x) else None
# )

housing.info()
housing.head()

<>:18: SyntaxWarning: invalid escape sequence '\$'
<>:18: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-961513799.py:18: SyntaxWarning: invalid escape sequence '\$'
  housing['price']=housing['price'].replace('[\$,]', '', regex=True).astype(float)


ValueError: could not convert string to float: 'Contact agent'

In [ ]:
#housing["region"].value_counts()

In [ ]:
housing["propertyType"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
housing['price']
